# Import modules

In [1]:
import getpass
import boto3
import pandas as pd
from impala.dbapi import connect
from impala.util import as_pandas
from io import BytesIO as StringIO

In [2]:
username = "abibeka@securedatacommons.com"
password1 = getpass.getpass()

········


In [3]:
conn = connect(host="172.18.1.20",auth_mechanism='PLAIN',port=10000,user=username, password=password1)

# Summary of Processed Data

In [4]:
cursor = conn.cursor()
start_date='2017-01-01'
end_date='2019-12-31' 
Q1='''SELECT max(FROM_UNIXTIME(CEILING(unix_timestamp(date_time,"MM/dd/yyyy hh:mm:ss")/300)*300,"yyyy-MM-dd hh:mm:ss")) AS MaxDate2018 FROM wydot_speed_processed
WHERE unix_timestamp(date_time,"MM/dd/yyyy hh:mm:ss") between unix_timestamp('{}',"yyyy-MM-dd") AND unix_timestamp('{}',"yyyy-MM-dd")'''.format(start_date,end_date)
cursor.execute(Q1)
TempDat1=as_pandas(cursor)
TempDat1.head()

,maxdate2018
0,2019-10-14 12:00:00


In [5]:
cursor = conn.cursor()
start_date='2016-01-01'
end_date='2019-12-31' 
Q1='''SELECT min(FROM_UNIXTIME(CEILING(unix_timestamp(date_time,"MM/dd/yyyy hh:mm:ss")/300)*300,"yyyy-MM-dd hh:mm:ss")) AS MinDate2018 FROM wydot_speed_processed
WHERE unix_timestamp(date_time,"MM/dd/yyyy hh:mm:ss") between unix_timestamp('{}',"yyyy-MM-dd") AND unix_timestamp('{}',"yyyy-MM-dd")'''.format(start_date,end_date)
cursor.execute(Q1)
TempDat1=as_pandas(cursor)
TempDat1.head()

,mindate2018
0,2017-01-01 01:00:00


In [6]:
cursor = conn.cursor()

Q1 = '''
SELECT * FROM wydot_speed_processed WHERE Sensor=396  LIMIT 100
'''
cursor.execute(Q1)
TempDat1=as_pandas(cursor)
TempDat1.head()

,wydot_speed_processed.date_time,wydot_speed_processed.sensor,wydot_speed_processed.speed,wydot_speed_processed.length,wydot_speed_processed.class,wydot_speed_processed.lane,wydot_speed_processed.rwis,wydot_speed_processed.wb_vsl,wydot_speed_processed.eb_vsl,wydot_speed_processed.sensor_loc,...,wydot_speed_processed.rh,wydot_speed_processed.surftemp,wydot_speed_processed.wndspd,wydot_speed_processed.stormnum,wydot_speed_processed.postedspd,wydot_speed_processed.postedspd_vsltime,wydot_speed_processed.speedcompliant5,wydot_speed_processed.speedbuffer10,wydot_speed_processed.dataquality,wydot_speed_processed.stormcat
0,09/29/2018 00:10:29,396,80.71,74.03,3,3,"""WY28""",3668,2118,"""EB""",...,2,1,1,10,75,"""09/26/2018 13:40:08""",0,1,"""1""",1
1,09/29/2018 00:10:33,396,72.27,20.72,2,3,"""WY28""",3668,2118,"""EB""",...,2,1,1,10,75,"""09/26/2018 13:40:08""",1,1,"""1""",1
2,09/29/2018 00:22:16,396,74.19,74.45,3,3,"""WY28""",3668,2118,"""EB""",...,1,1,1,1,75,"""09/26/2018 13:40:08""",1,1,"""1""",1
3,09/29/2018 00:22:30,396,82.77,20.31,2,3,"""WY28""",3668,2118,"""EB""",...,1,1,1,1,75,"""09/26/2018 13:40:08""",0,1,"""1""",1
4,09/29/2018 00:40:53,396,72.51,73.71,3,3,"""WY28""",3668,2118,"""EB""",...,1,1,1,1,75,"""09/26/2018 13:40:08""",1,1,"""1""",1


In [7]:
#Create a query
create_query = '''describe wydot_speed_processed '''
cursor.execute(create_query)
pandas_df = as_pandas(cursor)
pandas_df

,col_name,data_type,comment
0,date_time,string,
1,sensor,int,
2,speed,float,
3,length,float,
4,class,int,
5,lane,int,
6,rwis,string,
7,wb_vsl,int,
8,eb_vsl,int,
9,sensor_loc,string,


# Summary of Speed Sensors at Laramie and Cheyenne

In [8]:
start_date='2018-02-01'
end_date='2018-03-01' 
create_query='''
SELECT deviceid, sitename, direction, milepost, devicename, sensortype FROM wydot_speed_sensors_index WHERE deviceid = 396 OR deviceid = 1839 
'''
cursor.execute(create_query)
Spd_indx=as_pandas(cursor)
Spd_indx

,deviceid,sitename,direction,milepost,devicename,sensortype
0,396,Laramie East,B,317.68,Speed Sensor,Traffic
1,1839,Roundtop Interchange,B,356.70,Speed Sensor,Traffic


# Get 15 Volume Data by Lane for Feb 2018 

## Processed speed doesn't have data for Feb use Unprocessed Speed. Based on a Wydot excel file 396 is EB dir and 1839 is on WB dir

In [13]:
start_date='2018-02-01'
end_date='2018-03-01' 
deviceClause = '''
deviceid = 1327 OR deviceid = 1342 OR deviceid = 396 OR deviceid = 3911 OR deviceid =395 OR deviceid =2146 OR deviceid = 2319 OR deviceid = 1839 or deviceid  = 3482 OR deviceid = 382
'''
create_query='''SELECT t1.controller, t1.lane, t1.vehclass, t1.Time5M, t1.speedmph_avg, t1.NRec, t2.direction, t2.milepost, t2.2015_adt FROM 
    (SELECT controller, FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/900)*900) AS Time5M, lane, vehclass, AVG(speedmph) AS speedmph_avg, count(*) AS NRec
    FROM wydot_speed_unprocessed 
    WHERE mountain between '{}' AND '{}'
    GROUP BY controller, lane, vehclass,FROM_UNIXTIME(CEILING(UNIX_TIMESTAMP(mountain)/900)*900)) t1
JOIN (SELECT * FROM wydot_speed_sensors_index WHERE {}) t2 
ON (t1.controller = t2.deviceid)
ORDER BY t1.controller, t1.Time5M, t1.lane, t1.vehclass'''.format(start_date,end_date,deviceClause)
cursor.execute(create_query)
Vol_dat3=as_pandas(cursor)
#direction != 'B' AND

In [14]:
Vol_dat3.head()

,t1.controller,t1.lane,t1.vehclass,t1.time5m,t1.speedmph_avg,t1.nrec,t2.direction,t2.milepost,t2.2015_adt
0,382,1,1,2018-02-01 00:15:00,58.749999,3,B,401.8,8147
1,382,1,3,2018-02-01 00:15:00,48.120001,4,B,401.8,8147
2,382,2,1,2018-02-01 00:15:00,52.639999,1,B,401.8,8147
3,382,3,2,2018-02-01 00:15:00,61.349998,1,B,401.8,8147
4,382,3,3,2018-02-01 00:15:00,51.160000,1,B,401.8,8147


In [15]:
Vol_dat3.to_csv('Z:/Apoorb/Feb2018-FreevalVolDat.csv')